In [1]:
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#nltk.download('sentiwordnet')
import pandas as pd

In [2]:
pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
    --------------------------------------- 0.0/1.5 MB 1.3 MB/s eta 0:00:02
   - -------------------------------------- 0.0/1.5 MB 393.8 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.5 MB 939.4 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.5 MB 871.5 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.5 MB 1.0 MB/s eta 0:00:02
   -------- ------------------------------- 0.3/1.5 MB 1.2 MB/s eta 0:00:02
   ---------- ----------------------------- 0.4/1.5 MB 1.2 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.5 MB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 1.3 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 1.9 MB/s eta 0:00:01
   ----------


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# load the dataset
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
reviews = train_df["review"]
sentiment = train_df["sentiment"]

FileNotFoundError: [Errno 2] No such file or directory: '.data/train.csv'

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(texts, labels, train_size=0.75,random_state=1234)

In [ ]:
y_pred = []
for t in X_test:    
    pos_score = neg_score = token_count = obj_score = 0
    for word in t:        
        ss_set = None
        if len(list(swn.senti_synsets(word, 'n'))) > 0:        
            ss_set = list(swn.senti_synsets(word, 'n'))[0]        
        if ss_set:
            pos_score += ss_set.pos_score()
            neg_score += ss_set.neg_score()
            obj_score += ss_set.obj_score()
            token_count += 1
    final_score = pos_score - neg_score
    norm_final_score = round(float(final_score) / token_count, 2)
    final_sentiment = '2' if norm_final_score >= 0 else '1'
    y_pred.append(final_sentiment)
accuracy_score(y_test, y_pred)